In [1]:
class RefPath:
    INITIAL = "#/info/x-clusters/"
    WORKER_NODES = "worker-nodes"
    PODS = "pods"
    CONTAINERS = "containers"
    INFO = "info"
    X_CLUSTERS = "x-clusters"
    X_LOCATION = "x-location"
    REF = "$ref"
    PATHS = "paths"
    
    def __init__(self, *arg):
        if len(arg) == 1:            
            paths = arg[0].split("#/info/x-clusters/")
            parts = paths[1].split("/")        
            self.cluster = parts[0]
            self.worker_node = parts[2]
            self.pod_name = parts[4]
            self.container_name = parts[6]
        else:
            self.cluster = arg[0]
            self.worker_node = arg[1]
            self.pod_name = arg[2]
            self.container_name = arg[3]    
        

    @property
    def full_path(self):
        return f"{RefPath.INITIAL}{self.cluster}/{RefPath.WORKER_NODES}/{self.worker_node}/{RefPath.PODS}/{self.pod_name}/{RefPath.CONTAINERS}/{self.container_name}"


class Method:
    def __init__(self, path_name, method_name, ref_path, load, schema_name, full_method):
        self.path_name = path_name
        self.method_name = method_name
        self.ref_path = ref_path      
        self.load = load
        self.schema_name = schema_name        
        self.full_method = full_method
        self.contribution = 0

    def __str__(self):
        return self.path_name + ": " + self.method_name + " (" + self.ref_path.full_path + ") " + str(self.load) + " <-> " + self.schema_name
    
class Component:
    def __init__(self, name, load, full_component, ref_path=None):
        self.name = name
        self.load = load
        self.full_component = full_component
        self.ref_path = ref_path
        
    def __str__(self):
        return self.name + " ---- " + str(self.load) + " (" + self.ref_path + ")"
    
import operator
class Group:
    def __init__(self, parent_load, sum_of_loads, components):        
        self.parent_load = parent_load        
        self.sum_of_loads = sum_of_loads
        self.components = components
        
    def get_contributed_components(self):
        components = []
        for component in self.components:
            component.contribution = (component.load / self.sum_of_loads) * self.parent_load
            components.append(component)        
        return sorted(components, key=operator.attrgetter("contribution"))


In [2]:
class Cluster(Component):
    def __init__(self, pod_name, load, full_component, ref_path):        
        super().__init__(pod_name, load, full_component, ref_path)
        self.is_cluster = True
        
    @property
    def worker_nodes(self):
        worker_nodes = []
        for wn in self.full_component["worker-nodes"].values():
            ref_path = self.ref_path + "/worker-nodes/" + wn["name"]
            worker_nodes.append(WorkerNode(wn["name"], wn["metrics"]["load"], wn, ref_path))
        return worker_nodes
        
class WorkerNode(Component):
    def __init__(self, pod_name, load, full_component, ref_path):
        super().__init__(pod_name, load, full_component, ref_path)
        self.is_worker_node = True
        
    @property
    def pods(self):
        pods = []
        for pod in self.full_component["pods"].values():
            ref_path = self.ref_path + "/pods/" + pod["name"]
            pods.append(Pod(pod["name"], pod["metrics"]["load"], pod, ref_path))        
        return pods
        
class Pod(Component):
    def __init__(self, pod_name, load, full_component, ref_path):
        super().__init__(pod_name, load, full_component, ref_path)
        self.is_pod = True
        self.contribution = 0
        
    @property
    def containers(self):
        containers = []
        for container_name, container in self.full_component["containers"].items():
            ref_path = self.ref_path + "/containers/" + container_name
            containers.append(Container(container_name, container["metrics"]["load"], container, ref_path))        
        return containers
        
class Container(Component):
    def __init__(self, pod_name, load, full_component, ref_path):
        super().__init__(pod_name, load, full_component, ref_path)
        self.is_container = True


class PodGroup(Group):
    def __init__(self, parent_load, sum_of_loads, components):
        super().__init__(parent_load, sum_of_loads, components)
        self.is_pod_group = True            
        
    def __str__(self):
        return "PodGroup sum_of_pod_loads: " + str(sum_of_pod_loads)
    
class ContainerGroup(Group):
    def __init__(self, parent_load, sum_of_loads, components):
        super().__init__(parent_load, sum_of_loads, components)
        self.is_container_group = True            
        
    def __str__(self):
        return "ContainerGroup sum_of_pod_loads: " + str(sum_of_pod_loads)
    
class MethodGroup(Group):
    def __init__(self, parent_load, sum_of_loads, components):
        super().__init__(parent_load, sum_of_loads, components)
        self.is_method_group = True            
        
    def __str__(self):
        return "ContainerGroup sum_of_pod_loads: " + str(sum_of_pod_loads)
        

In [3]:
def _gen_dict_extract(key, var):
    if hasattr(var,'items'):
        for k, v in var.items():
            if k == key:
                yield v
            if isinstance(v, dict):                
                for result in _gen_dict_extract(key, v):
                    yield result
            elif isinstance(v, (list)):
                for d in v:
                    for result in _gen_dict_extract(key, d):
                        yield result                        


def _get_schema_only(references):
    saved_schema = 'default'
    for reference in references:
        schema = reference.split("#/components/schemas/")        
        try:
            saved_schema = schema[1]
        except IndexError:
            pass
    
    return saved_schema


def _join_components(threshold, components):    
    joined_components = []
    remaining_components = []
    collective_contribution = 0
    for i in range(len(components)):
        component = components[i]
        collective_contribution += component.contribution
        
        if collective_contribution < threshold:
            joined_components.append(component)
        else:
            remaining_components.append(component)
        
    return joined_components, remaining_components

In [11]:
group.get_contributed_components()
for comp in group.get_contributed_components():
    print(comp, comp.contribution)
    
_join_components(10, group.get_contributed_components())

pod1 ---- 51 (#/info/x-clusters/cl1/worker-nodes/wn1/pods/pod1) 18.461538461538463
pod2 ---- 80 (#/info/x-clusters/cl1/worker-nodes/wn1/pods/pod2) 28.959276018099548
pod3 ---- 90 (#/info/x-clusters/cl1/worker-nodes/wn1/pods/pod3) 32.57918552036199


([],
  <__main__.Pod at 0x7fb6c59142b0>])

In [4]:
# Monitor Scaling start from here
import json

with open("01data.json") as datafile:
    all_data = json.load(datafile)
    

with open("01config.json") as config_file:
    config_template = json.load(config_file)

In [10]:
import copy

MAX_WN_LOAD = 50

copied_template = copy.deepcopy(config_template)
for single_data_obj in all_data:
    clusters = []
    data_paths = single_data_obj["paths"]
    cls = single_data_obj["x-clusters"]
    
    for cl in cls.values():
        ref_path = RefPath.INITIAL + cl["name"]
        clusters.append(Cluster(cl["name"], cl["metrics"]["load"], cl, ref_path))
    
    methods = []
    for path_name, path in data_paths.items():
        for method_name, method in path.items():
            ref_path = RefPath(method["x-location"]["$ref"])

            all_references = list(set(_gen_dict_extract('$ref', method)))
            schema_name = _get_schema_only(all_references)

            methods.append(Method(path_name, method_name, ref_path, method["x-metrics"]["load"], schema_name, method))
            
    scalable_wns = []
#     triggering the scaling by workernode threshold exceed
    for cluster in clusters:        
        for worker_node in cluster.worker_nodes:
            if worker_node.load < MAX_WN_LOAD:
                print("no need to scale ", str(worker_node))                
                continue
            else:
                print(worker_node, " need scaling.")
                scalable_wns.append(worker_node)
    
    pod_groups = []
    for wn in scalable_wns:
        sum_of_pod_loads = sum(p.load for p in wn.pods)
        pod_groups.append(PodGroup(wn.load, sum_of_pod_loads, wn.pods))
    
    
    for group in pod_groups:
        joined_pods, remaining_pods = _join_components(MAX_WN_LOAD, group.get_contributed_components())
        
#         getting first pod from remaining pods
        next_joining_pod = remaining_pods[0]        
        sum_of_container_loads = sum(c.load for c in next_joining_pod.containers)
        container_group = ContainerGroup(next_joining_pod.load, sum_of_container_loads, next_joining_pod.containers)
        
        sum_of_joined_pods_contribution = sum(j_p.contribution for j_p in joined_pods)        
        joined_containers, remaining_containers = _join_components(
            (MAX_WN_LOAD - sum_of_joined_pods_contribution), container_group.get_contributed_components())
        
#         getting first container from remaining containers
        next_joining_container = remaining_containers[0]
        
        next_joining_container_methods = []
        for method in methods:            
            if method.ref_path.full_path == next_joining_container.ref_path:
                next_joining_container_methods.append(method)
        sum_of_method_loads = sum(m.load for m in next_joining_container_methods)
        method_group = MethodGroup(next_joining_container.load, sum_of_method_loads, next_joining_container_methods)
        sum_of_joined_containers_contribution = sum(j_c.contribution for j_c in joined_containers)
        joined_methods, remaining_methods = _join_components(
            MAX_WN_LOAD - sum_of_joined_pods_contribution - sum_of_joined_containers_contribution,
            method_group.get_contributed_components())                
                
        
        new_worker_node = None
        if remaining_pods or remaining_containers or remaining_methods:            
            wn_name = "wn" + str(len(cluster.worker_nodes) + 1)
            print("create a new WN", wn_name)
            
            new_worker_node = {                
                "name": wn_name, 
                 "metrics": {"load": ""},
                 "pods": {}
                }            
            
            
        new_pod = None
        if remaining_methods or remaining_containers[1:]:            
            pod_name = "pod" + str(len(remaining_pods[1:]) + 1)
            print("creating a new POD", pod_name)
            new_pod = {
                    "name": "pod1",
                    "metrics": {
                      "load": ""
                    },"containers": {}
            }
        
#         it means that all containers in first pod needs to get in new WN
        remaining_pods_to_deploy = remaining_pods if len(joined_containers) == 0 else remaining_pods[1:]
        
        if remaining_pods_to_deploy:
            print("create a new worker-node and add the pods in it.")
            
        remaining_containers_to_deploy = remaining_containers if len(joined_methods) == 0 else remaining_containers[1:]
        if remaining_containers_to_deploy:            
            print("create a new pod and add the containers in it.")
            for r_container in remaining_containers_to_deploy:
                r_container_ref_path = RefPath(r_container.ref_path)
                r_container_ref_path.pod_name = pod_name
                r_container_ref_path.worker_node = wn_name
                                
#                 replacing the path of methods
                r_container_methods = []
                for method in methods:                    
                    if method.ref_path.full_path == r_container.ref_path:                        
                        method.ref_path = r_container_ref_path
                        ind_method = copied_template[RefPath.PATHS][method.path_name][method.method_name]
                        ind_method[RefPath.X_LOCATION][RefPath.REF] = ref_path.full_path            
                        
                        print(method)
                
#                 also replacing the path of container
                r_container.ref_path = r_container_ref_path.full_path
        
        cluster_template = copied_template[RefPath.INFO][RefPath.X_CLUSTERS][cluster.name][RefPath.WORKER_NODES]
        
        if new_worker_node:
            cluster_template[wn_name] = new_worker_node
            if new_pod:
                cluster_template[wn_name][RefPath.PODS][pod_name] = new_pod
        
        new_container = None        
        if remaining_methods:
            print("create a new container and add the methods in it")
            container_name = "c" + str(len(remaining_containers[1:]) + 1)
            new_container = {
                    "name": container_name,
                    "metrics": {
                      "load": ""
                    }
                  }
            
            ref_path = RefPath(cluster.name, new_worker_node["name"], new_pod["name"], new_container["name"])
            cluster_template[wn_name][RefPath.PODS][pod_name][RefPath.CONTAINERS][container_name] = new_container
        
#         changing the RefPath of remaining methods
        for r_method in remaining_methods:            
            method = copied_template[RefPath.PATHS][r_method.path_name][r_method.method_name]
            method[RefPath.X_LOCATION][RefPath.REF] = ref_path.full_path            
                    

copied_template

wn1 ---- 80 (#/info/x-clusters/cl1/worker-nodes/wn1)  need scaling.
create a new WN wn2
creating a new POD pod1
create a new worker-node and add the pods in it.
create a new pod and add the containers in it.
/pets: post (#/info/x-clusters/cl1/worker-nodes/wn2/pods/pod1/containers/c1) 92 <-> Pet
/pets: get (#/info/x-clusters/cl1/worker-nodes/wn2/pods/pod1/containers/c2) 78 <-> default


{'openapi': '3.0.0',
 'info': {'description': 'This is a sample server Petstore server.  You can find out more about     Swagger at [http://swagger.io](http://swagger.io) or on [irc.freenode.net, #swagger](http://swagger.io/irc/).      For this sample, you can use the api key `special-key` to test the authorization     filters.',
  'version': '1.0.0',
  'title': 'Swagger Petstore',
  'termsOfService': 'http://swagger.io/terms/',
  'contact': {'email': 'apiteam@swagger.io'},
  'license': {'name': 'Apache 2.0',
   'url': 'http://www.apache.org/licenses/LICENSE-2.0.html'},
  'x-clusters': {'cl1': {'name': 'cl1',
    'metrics': {'load': ''},
    'worker-nodes': {'wn1': {'metrics': {'load': ''},
      'name': 'wn1',
      'pods': {'pod1': {'name': 'pod1',
        'metrics': {'load': ''},
        'containers': {'c1': {'id': 'c1', 'metrics': {'load': ''}},
         'c2': {'id': 'c2', 'metrics': {'load': ''}},
         'c3': {'id': 'c3', 'metrics': {'load': ''}},
         'c4': {'id': 'c4', 'm

In [10]:
MAX_WN_LOAD = 30
single_data_obj = all_data[0]
clusters = []
data_paths = single_data_obj["paths"]
cls = single_data_obj["x-clusters"]

for cl in cls.values():
    ref_path = RefPath.INITIAL + cl["name"]
    clusters.append(Cluster(cl["name"], cl["metrics"]["load"], cl, ref_path))

methods = []
for path_name, path in data_paths.items():
    for method_name, method in path.items():
        ref_path = RefPath(method["x-location"]["$ref"])

        all_references = list(set(_gen_dict_extract('$ref', method)))
        schema_name = _get_schema_only(all_references)

        methods.append(Method(path_name, method_name, ref_path, method["x-metrics"]["load"], schema_name, method))

scalable_wns = []
#     triggering the scaling by workernode threshold exceed
for cluster in clusters:        
    for worker_node in cluster.worker_nodes:
        if worker_node.load < MAX_WN_LOAD:
            print("no need to scale ", str(worker_node))                
            continue
        else:
            print(worker_node, " need scaling.")
            scalable_wns.append(worker_node)

pod_groups = []
for wn in scalable_wns:
    sum_of_pod_loads = sum(p.load for p in wn.pods)
    pod_groups.append(PodGroup(wn.load, sum_of_pod_loads, wn.pods))


for group in pod_groups:
    joined_pods, remaining_pods = _join_components(MAX_WN_LOAD, group.get_contributed_components())
    
#         getting first pod from remaining pods
    next_joining_pod = remaining_pods[0]        
    sum_of_container_loads = sum(c.load for c in next_joining_pod.containers)
    container_group = ContainerGroup(next_joining_pod.load, sum_of_container_loads, next_joining_pod.containers)

    sum_of_joined_pods_contribution = sum(j_p.contribution for j_p in joined_pods)        
    joined_containers, remaining_containers = _join_components(
        (MAX_WN_LOAD - sum_of_joined_pods_contribution), container_group.get_contributed_components())

#         getting first container from remaining containers
    next_joining_container = remaining_containers[0]

    next_joining_container_methods = []
    for method in methods:            
        if method.ref_path.full_path == next_joining_container.ref_path:
            next_joining_container_methods.append(method)
    sum_of_method_loads = sum(m.load for m in next_joining_container_methods)
    method_group = MethodGroup(next_joining_container.load, sum_of_method_loads, next_joining_container_methods)
    sum_of_joined_containers_contribution = sum(j_c.contribution for j_c in joined_containers)
    joined_methods, remaining_methods = _join_components(
        MAX_WN_LOAD - sum_of_joined_pods_contribution - sum_of_joined_containers_contribution,
        method_group.get_contributed_components())                


wn1 ---- 80 (#/info/x-clusters/cl1/worker-nodes/wn1)  need scaling.


In [43]:
# getting all methods from a fat container

methods = []
for path_name, path in initial_template["paths"].items():
    for method_name, method in path.items():        
        ref_path = RefPath(method["x-location"]["$ref"]) 
        
        all_references = list(set(gen_dict_extract('$ref', method)))        
        schema_name = get_schema_only(all_references)            
        
        methods.append(Method(path_name, method_name, ref_path, schema_name, method))

pods = []
for pod_name, pod in initial_template["info"]["x-pods"].items():
    full_pod = dict()
    full_pod[pod_name] = pod
    pods.append(Pod(pod_name, full_pod))

In [50]:
pods[0].full_pod[pod_name]["containers"]

{'c1': {'id': 'c1', 'metrics': {'load': ''}, 'port': 4000}}

In [73]:
schema_grouped_methods = {}
for method in methods:  
    schema_grouped_methods.setdefault(method.schema_name, [])
    schema_grouped_methods[method.schema_name].append(method)    

In [1]:
import copy

copied_template = copy.deepcopy(initial_template)

index = 1
for schema_name, schema_methods in schema_grouped_methods.items():
    if index == 1:
        pass        
    else:
        first_method = schema_methods[0]
        new_container = "c" + str(index)
        
        container_template = {'id': new_container, 'metrics': {'load': ''} }
        
        copied_template["info"]["x-pods"][first_method.ref_path.pod_name]["containers"][new_container] = container_template
        
        for method in schema_methods:
            method.ref_path.container_name = new_container
            method.full_method["x-location"]["$ref"] = method.ref_path.full_path
            copied_template["paths"][method.path_name][method.method_name] = method.full_method
    index +=1
    


NameError: name 'initial_template' is not defined

In [74]:
class SchemaGroup:
    def __init__(self, schema_name, sum_of_loads, methods):
        self.schema_name = schema_name
        self.sum_of_loads = sum_of_loads
        self.methods = methods
        
    def __str__(self):
        return schema_group.schema_name + "-" + str(schema_group.sum_of_loads)

In [76]:
MAX_ENDPOINT_LOAD = 60
MAX_CPU_USAGE = 75
MAX_RAM_USAGE = 70

MIN_ENDPOINT_LOAD = 25
MIN_CPU_USAGE = 40
MIN_RAM_USAGE = 40

scalable_schemas = []
methods = []

for single_data_obj in all_data:
    data_paths = single_data_obj["paths"]
    data_pods = single_data_obj["x-pods"]

    number_of_pods = len(data_pods)

    methods = []
    for path_name, path in data_paths.items():
        for method_name, method in path.items():
            ref_path = RefPath(method["x-location"]["$ref"])

            all_references = list(set(_gen_dict_extract('$ref', method)))
            schema_name = _get_schema_only(all_references)

            methods.append(Method(path_name, method_name, ref_path, method["x-metrics"]["load"], schema_name, method))

    for method in methods:
        if MIN_ENDPOINT_LOAD < method.load <= MAX_ENDPOINT_LOAD:
#             print("No Need of change {" + method.__str__() + "}")
            continue
        else:
#             print("Load is demanding to change the config {" + method.__str__() + "}")
            scalable_schemas.append(method.schema_name)
            
    scalable_schemas = list(set(scalable_schemas))
    if scalable_schemas:
        break;


schema_groups = []
for scalable_schema in scalable_schemas:
    schema_group = SchemaGroup(scalable_schema, 0, list())
    for method in methods:
        if method.schema_name == scalable_schema:            
            schema_group.sum_of_loads += method.load
            schema_group.methods.append(method)
    
    schema_groups.append(schema_group)

            

import operator

schema_groups = sorted(schema_groups, key=operator.attrgetter("sum_of_loads"))

for schema_group in schema_groups:
    schema_group.methods = sorted(schema_group.methods, key=operator.attrgetter("load"))

    
for schema_group in schema_groups:
    print(schema_group)
    for method in schema_group.methods:
        print(method)

User-12
/users: patch (#/info/x-pods/pod1/containers/c3/port) 12 <-> User
Store-83
/store: post (#/info/x-pods/pod1/containers/c4/port) 83 <-> Store
Pet-118
/pets: put (#/info/x-pods/pod1/containers/c2/port) 31 <-> Pet
/store: patch (#/info/x-pods/pod1/containers/c2/port) 87 <-> Pet
default-335
/pets: post (#/info/x-pods/pod1/containers/c1/port) 74 <-> default
/pets: get (#/info/x-pods/pod1/containers/c1/port) 82 <-> default
/users: get (#/info/x-pods/pod1/containers/c1/port) 86 <-> default
/store: get (#/info/x-pods/pod1/containers/c1/port) 93 <-> default
